# Define Parameters and Connections

## Define stored procedure, sales channel id, and input table

In [ ]:
import os

stored_proc_name = jdbcHostname = os.getenv("SP_NAME")
sales_channel_id = jdbcHostname = os.getenv("CHANNEL_ID")
input_table_name = jdbcHostname = os.getenv("TABLE_NAME")

## Define yesterday timestamp

In [ ]:
from datetime import date, datetime, timedelta, timezone
import pytz

# time zone
tz = 'US/Pacific'
# difference between current and previous date
delta = timedelta(days=1) # change to days=1 in production.

# define yesterday
yesterday = datetime.now(pytz.utc).astimezone(pytz.timezone(tz)) - delta
yesterday_str = yesterday.strftime('%Y-%m-%d')

In [ ]:
yesterday, yesterday_str

(datetime.datetime(2024, 1, 21, 13, 37, 34, 943235, tzinfo=<DstTzInfo 'US/Pacific' PST-1 day, 16:00:00 STD>),
 '2024-01-21')

## Define Azure SQL DB connnection

In [ ]:
import os

# Define Azure SQL Database connection
jdbcHostname = os.getenv("SQLDB_HOST")
user = os.getenv("SQLDB_USER")
password = dbutils.secrets.get(scope="azure_key_vault", key="SQLDB-PW") # use Azure Key Vault to save this password. 
jdbcDatabase = os.getenv("SQLDB_BBDW")
jdbcPort = 1433
jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostname, jdbcPort, jdbcDatabase)
connectionProperties = {
"user" : user,
"password" : password,
"driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

## Define NetSuite REST connnection

Each request must use unique nonce. Therefore, it needs to create a random nonce for each POST request.

In [ ]:
import requests
from requests_oauthlib import OAuth1
import time
import random
import hashlib
import hmac
import base64
import urllib.parse

consumer_key = dbutils.secrets.get(scope="azure_key_vault", key="NS-CONSUMER-KEY") # use Azure Key Vault to save this. 
consumer_secret = dbutils.secrets.get(scope="azure_key_vault", key="NS-CONSUMER-SECRET") # use Azure Key Vault to save this. 
access_token = dbutils.secrets.get(scope="azure_key_vault", key="NS-TOKEN-ID") # use Azure Key Vault to save this. 
token_secret = dbutils.secrets.get(scope="azure_key_vault", key="NS-TOKEN-SECRET") # use Azure Key Vault to save this. 
realm = os.getenv("NS_REALM") # change to NS_REALM in production.

# The URL you want to access
url = "https://{0}.suitetalk.api.netsuite.com/services/rest/record/v1/creditMemo/".format(os.getenv("NS_ACCOUNT_ID")) # change to NS_ACCOUNT_ID in production.

# The HTTP method you want to use
method = "POST"

def generateHeader(consumer_key, consumer_secret, access_token, token_secret, realm, url, method):
    # Generate a nonce, which is a random string that is unique for each request
    nonce = "".join(random.choice("abcdefghijklmnopqrstuvwxyz0123456789") for i in range(32))

    # Generate a timestamp, which is the number of seconds since January 1, 1970
    timestamp = str(int(time.time()))

    # Create a dictionary with the OAuth parameters
    oauth_params = {
        "oauth_consumer_key": consumer_key,
        "oauth_token": access_token,
        "oauth_signature_method": "HMAC-SHA256",
        "oauth_timestamp": timestamp,
        "oauth_nonce": nonce,
        "oauth_version": "1.0"
    }

    # Create a signature base string, which is a concatenation of the HTTP method, the request URL, and the OAuth parameters
    base_string = method.upper() + "&" + urllib.parse.quote(url, safe="") + "&" + urllib.parse.quote("&".join(sorted(key + "=" + urllib.parse.quote(value, safe="") for key, value in oauth_params.items())), safe="")

    # Create a signing key, which is a combination of the consumer secret and the token secret
    signing_key = urllib.parse.quote(consumer_secret, safe="") + "&" + urllib.parse.quote(token_secret, safe="")

    # Calculate a signature, which is a hash of the signature base string and the signing key using the specified signature method
    signature = base64.b64encode(hmac.new(signing_key.encode(), base_string.encode(), hashlib.sha256).digest()).decode()

    # Add the signature to the OAuth parameters
    oauth_params["oauth_signature"] = signature

    # Construct the Authorization header, which is a string that contains the OAuth parameters and the signature
    auth_header = "OAuth realm=\"" + realm + "\"," + ",".join(key + "=\"" + urllib.parse.quote(value, safe="") + "\"" for key, value in oauth_params.items())

    # Create a dictionary with the Authorization key and value
    headers = {"Authorization": auth_header}

    return headers

# Execute Stored Procedure to generate the data input

In [ ]:
driver_manager = spark._sc._gateway.jvm.java.sql.DriverManager
connection = driver_manager.getConnection(jdbcUrl, user, password)
query = "EXEC {0} '{1}', {2}, null;".format(stored_proc_name, yesterday_str, sales_channel_id)
connection.prepareCall(query).execute()
connection.close()

In [ ]:
# Load data from Azure SQL Database into a dataframe
df = spark.read.jdbc(
    url=jdbcUrl,
    table=input_table_name,
    properties=connectionProperties,
)

# Prepare a JSON Body and POST a credit memo on NetSuite for each customer

In [ ]:
from pyspark.sql.functions import collect_list, struct, explode

struct_df = df.groupBy("customer_id").agg(collect_list(struct(*df.columns)).alias("struct_data"))

for id in struct_df.select("customer_id").collect():
    customer_df = struct_df.select("struct_data").where("customer_id == {0}".format(id["customer_id"]))
    customer_df = customer_df.select(explode("struct_data").alias("col_name"))
    customer_df = customer_df.select("col_name.*")

    attributes = ["customer", "department", "location", "subsidiary"]
    for attribute in attributes:
        attribute_dict = {}
        column = f"{attribute}_id"
        attribute_dict["id"] = customer_df.select(column).distinct().collect()[0][column]
        exec(f"{attribute}_dict = {attribute_dict}")

    item_list = []
    item_cols = ["line_internal_id", "quantity", "rate", "default_bin"]
    for item in customer_df.select(item_cols).collect():
        if not item["default_bin"]:
            item_dict = {
                "item": {"id": item["line_internal_id"]},
                "quantity": item["quantity"],
                "rate": float(item["rate"]),
                "price": {"id": "-1"},
                "commitInventory": {"id": "1"},
            }
        else:  
            item_dict = {
                "item": {"id": item["line_internal_id"]},
                "quantity": item["quantity"],
                "rate": float(item["rate"]),
                "price": {"id": "-1"},
                "commitInventory": {"id": "1"},
                "inventoryDetail": {
                    "inventoryAssignment": {
                        "items": [
                            {
                                "binNumber": {
                                    "id": item["default_bin"]
                                },
                                "quantity": item["quantity"]
                            }
                        ]
                    }
                }
            }
        item_list.append(item_dict)

    json_body = {
        "entity": customer_dict,  # customer
        "createdDate": customer_df.select("createddate").distinct().collect()[0]["createddate"] + "T00:00:00Z",  # It requires timestamp after the date.
        "tranDate": customer_df.select("createddate").distinct().collect()[0]["createddate"],
        "otherRefNum": customer_df.select("otherrefnum").distinct().collect()[0]["otherrefnum"],
        "memo": customer_df.select("memomain").distinct().collect()[0]["memomain"],
        "department": department_dict,
        "location": location_dict,
        "item": {"items": item_list},
        "subsidiary": subsidiary_dict,
        "status": {"id": "Open"}
    }
    # Make the request and print the response
    response = requests.post(url, headers=generateHeader(consumer_key, consumer_secret, access_token, token_secret, realm, url, method), json=json_body)
    # print(response.status_code)
    # print(response.content)
    # print(json_body)